In [15]:
import pickle
import numpy as np
import pandas as pd
from time import time
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.metrics import roc_curve, auc
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

/usr/local/lib/python2.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [18]:
with open('NHlegoverall.dict','rb') as f:
    leg_data = pickle.load(f)
with open('NHLegTimeSeriesLabels.list','rb') as f:
    leg_target = pickle.load(f)

ValueError: unsupported pickle protocol: 3

In [ ]:
print(leg_data.keys())
print(leg_data['sponsor_vec2'][100:105])


In [ ]:
doc_embeddings = pd.read_pickle("NHlegtext_embed_wBill.DF")
#print(doc_embeddings)
doc_embeddings.describe()

new_leg_data = {'event':leg_data['event'], 'bill_cat':leg_data['bill_cat'], 'chamber':leg_data['chamber'],
                'sponsor_vec':leg_data['sponsor_vec2'], 'time_t':leg_data['time_t'], 'year':leg_data['year'],
                'session':leg_data['session']}

#normalize the time data
for i, value in enumerate(new_leg_data['time_t']):
    new_leg_data['time_t'][i] = value/10 

#doesnt make a difference whether we make sessions and year categorical data instead of integers or not, so don't do it
# for i, value in enumerate(new_leg_data['session']):
#     new_leg_data['session'][i] = new_leg_data['session'][i]
#     new_leg_data['year'][i] = new_leg_data['year'][i]

#transform sponsor_vec to between 0 and 1
#for i, value in enumerate(new_leg_data['sponsor_vec']):
#    new_leg_data['sponsor_vec'][i] = 0.5*new_leg_data['sponsor_vec'][i] + 0.5

    
#set the default embedding to the average embedding from df.describe()
#default_embedding = {'doc_id':0, 'bill_id':0, 'dv1':-0.429835,'dv2':-0.348652,'dv3':-0.096175,'dv4':0.722217,'dv5':-0.080119,'dv6':0.331679,'dv7':0.886933,'dv8':0.934954,'dv9':-0.798512,'dv10':0.054533,'dv11':-0.028231,'dv12':0.561587,'dv13':-0.485424,'dv14':0.015666,'dv15':0.864729,'dv16':-0.697787}


leg_data = new_leg_data


In [ ]:
leg_data_df = pd.DataFrame(leg_data)
leg_data_df = pd.get_dummies(leg_data_df)
leg_label_df = pd.DataFrame(leg_target)
leg_label_df = pd.get_dummies(leg_label_df)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(leg_data_df, leg_target, test_size=0.3,random_state=109, shuffle=False) # 70% training and 30% test

In [ ]:
#define a performance metrics function (accuracy and percision and recall)
def perf_metric (y_test, y_pred):
#Accuracy
    x=0
    y=0
    acc=0
    for i in range(10205):
        if y_test[i] == y_pred[i]:
            x+=1
        y+=1
    acc=x/y*100
    print(x,y,len(y_pred),len(y_test))
    print('Accuracy:', acc)

#Precision and Recall
    tp=0
    fp=0
    fn=0
    tn=0
    perc=0 
    recall= 0 
        
    for i in range(10205):
        if y_pred[i]==1 and y_test[i]==1:
            tp+=1
        elif y_pred[i]==1 and y_test[i]==0:
            fp+=1
        elif y_pred[i]==0 and y_test[i]==1:
            fn+=1
        elif y_pred[i]==0 and y_test[i]==0:
            tn+=1
        
    if tp==0 and fp==0: 
        perc=0
    else:
        perc= tp/(tp+fp)  
          
    if tp==0 and fn==0: 
        recall=0
    else: 
        recall = tp/(tp+fn)

#print(tp,fp,fn,tn)
    print('Precision:', perc)
    print('Recall', recall)

In [ ]:
#####################################  DecisionTreeClassifier #################################

In [ ]:
tree = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort=False)

#Train the model using the training sets
tree.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = tree.predict(X_test)
y_proba = tree.predict_proba(X_test)
#y_proba_log = tree.predict_log_proba(X_test)

In [ ]:
perf_metric(y_test, y_pred)

In [ ]:
#####################################  Gaussian Classifier #################################

In [ ]:
#Create a Gaussian Classifier
mnnb = MultinomialNB()

#Train the model using the training sets
mnnb.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = mnnb.predict(X_test)
y_proba = mnnb.predict_proba(X_test)
y_proba_log = mnnb.predict_log_proba(X_test)

In [ ]:
perf_metric(y_test,y_pred)

In [ ]:
#####################################  Multinomial Classifier #################################

In [ ]:
#Note/ Question: does not work because of the negative values in Sponsor Vec
from sklearn.naive_bayes import MultinomialNB
mnnb = MultinomialNB()

#Train the model using the training sets
mnnb.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = mnnb.predict(X_test)
y_proba = mnnb.predict_proba(X_test)
y_proba_log = mnnb.predict_log_proba(X_test)

In [ ]:
perf_metric(y_test,y_pred)

In [ ]:
##################################### Random Forest #################################

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(X_train, y_train)  
y_pred= clf.predict(X_test)

In [3]:
perf_metric(y_test,y_pred)

NameError: name 'perf_metric' is not defined

In [4]:
##################################### Neural Network #################################

In [5]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier()
clf.fit(X_train, y_train)                         

y_pred=clf.predict(X_test)

NameError: name 'X_train' is not defined

In [147]:
perf_metric(y_test,y_pred)

927 10205 10205 10205
Accuracy: 9.083782459578638
Precision: 0
Recall 0
